In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import r2_score
import pickle
import time
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize
from scipy import sparse
import os.path
from scipy.interpolate import splrep, splev
from importlib import reload
import pandas as pd

In [3]:
import sys
sys.path.append('../.')
from data import input as input
from algorithms import algo as algo
from algorithms import utility as utility 
from model import anomaly as anomaly
from algorithms import robust_pca as robust_pca
from tqdm import tqdm

In [29]:
#%pdb
reload(input)
reload(algo)
reload(utility)
reload(anomaly)
reload(robust_pca)

def generate_data_for_table_cost(num_exper=200, r_range = [1, 10], mean_value_range = [1, 10], 
                                 p_o_range = [0.5, 1], p_a_range = [0, 0.3], alpha_range = [0, 1], metrics = None):

    parameters = []
    results = []
    start = 0
    n1 = 100
    n2 = 100
    for T in tqdm(range(start, num_exper)):
        r = np.random.randint(r_range[0], high=r_range[1]+1)
        mean_value = np.random.uniform(mean_value_range[0], mean_value_range[1])
        p_o = np.random.uniform(p_o_range[0], p_o_range[1])
        p_a = np.random.uniform(p_a_range[0], p_a_range[1])
        alpha = np.random.uniform(alpha_range[0], alpha_range[1])

        data = input.DataInput()
        data.synthetic_data(n1=n1, n2=n2, r=r, mean_value = mean_value, prob_observing = p_o)
        anomaly_model = anomaly.ExponentialAnomaly(p = p_a, exp_rate = 1/alpha, p_range = (0.01, 0.5), one_over_exp_range=(0.01, 0.9))
        data.add_anomaly(anomaly_model)
        data.cost_generation(c01 = [0, 10], c10 = [0, 10])
        data.compute_posterior()

        result_dict = {}
        result_dict['c_opt'] = utility.ideal_cost(data)

        result_dict.update(metrics(*algo.EW_algorithm(data, r_constraint = r, debug=False,  hard_impute_yes=False, gamma = data.n1/r*0.5), 'EW', data))

        #print(c_opt[T], c_EW[T])
        result_dict.update(metrics(*algo.DRMF(data, r, rate_anomalies=p_a), 'DRMF', data))
        
        result_dict.update(metrics(*robust_pca.Robust_PCA(data, r_constraint=r), 'stable_PCP', data))

        result_dict.update(metrics(*robust_pca.Robust_PCA(data, r_constraint=r, up_M = np.max(data.M0)*10, up_A = np.max(data.M0)*10), 'RMC', data))

        results.append(result_dict)
        parameters.append((r, mean_value, p_o, p_a, alpha))

        print(result_dict)

    result_df = pd.DataFrame.from_records(results)
    return result_df, parameters

def metrics(M_est, anomaly_decision, algo_name, data):
    results = {
        f'c_{algo_name}': utility.cost_compute(data, anomaly_decision),
        f'frobenious_norm_{algo_name}': utility.error_Frobenious(M_est, data.M0),
        f'max_norm_{algo_name}': utility.error_max(M_est, data.M0)
    }
    return results

results_df, parameters = generate_data_for_table_cost(num_exper=100, metrics = metrics)

  1%|          | 1/100 [00:01<02:05,  1.27s/it]

{'c_opt': 0.05449461405418702, 'c_EW': 0.07200582519253833, 'frobenious_norm_EW': 244.08877634224206, 'max_norm_EW': 25.559035784203274, 'c_DRMF': 0.45989252618511034, 'frobenious_norm_DRMF': 250.47937482887838, 'max_norm_DRMF': 33.15314993649032, 'c_stable_PCP': 0.5804028067575869, 'frobenious_norm_stable_PCP': 358.728904659007, 'max_norm_stable_PCP': 30.428378751084256, 'c_RMC': 0.5804028067575869, 'frobenious_norm_RMC': 358.728904659007, 'max_norm_RMC': 30.428378751084256}


  2%|▏         | 2/100 [00:13<12:28,  7.64s/it]

{'c_opt': 0.3051144503213981, 'c_EW': 0.3297892576012921, 'frobenious_norm_EW': 167.8358328858621, 'max_norm_EW': 22.060631681994977, 'c_DRMF': 0.6436971323773722, 'frobenious_norm_DRMF': 273.6653155087556, 'max_norm_DRMF': 36.091275906588535, 'c_stable_PCP': 0.5661977814623229, 'frobenious_norm_stable_PCP': 197.50018373074397, 'max_norm_stable_PCP': 27.743972348419543, 'c_RMC': 0.5661977814623229, 'frobenious_norm_RMC': 197.50018373074397, 'max_norm_RMC': 27.743972348419543}


  3%|▎         | 3/100 [00:20<11:43,  7.25s/it]

...

{'c_opt': 0.3714767999137694, 'c_EW': 0.4080928546997741, 'frobenious_norm_EW': 195.4488248481748, 'max_norm_EW': 16.83388051688353, 'c_DRMF': 0.737203864697278, 'frobenious_norm_DRMF': 171.76563439018372, 'max_norm_DRMF': 31.290256095476416, 'c_stable_PCP': 0.8878403720714357, 'frobenious_norm_stable_PCP': 354.023381964948, 'max_norm_stable_PCP': 33.76551246730547, 'c_RMC': 0.8878403720714357, 'frobenious_norm_RMC': 354.023381964948, 'max_norm_RMC': 33.76551246730547}


 99%|█████████▉| 99/100 [34:25<00:10, 10.38s/it]

{'c_opt': 0.3492630354003682, 'c_EW': 0.40697695184695126, 'frobenious_norm_EW': 243.18070042488097, 'max_norm_EW': 47.238686462051504, 'c_DRMF': 0.999451412290935, 'frobenious_norm_DRMF': 435.8809822219323, 'max_norm_DRMF': 74.65767410592022, 'c_stable_PCP': 0.9912193646511115, 'frobenious_norm_stable_PCP': 419.23184583559123, 'max_norm_stable_PCP': 62.30605669270209, 'c_RMC': 0.9912193646511115, 'frobenious_norm_RMC': 419.23184583559123, 'max_norm_RMC': 62.30605669270209}


100%|██████████| 100/100 [34:28<00:00, 20.69s/it]

{'c_opt': 0.2059099524830689, 'c_EW': 0.21570087890579343, 'frobenious_norm_EW': 179.69259185012268, 'max_norm_EW': 37.859044686499715, 'c_DRMF': 0.42276038212452977, 'frobenious_norm_DRMF': 136.53936550144508, 'max_norm_DRMF': 24.020729393732665, 'c_stable_PCP': 0.6206242604304331, 'frobenious_norm_stable_PCP': 441.82225512379597, 'max_norm_stable_PCP': 101.41442270181936, 'c_RMC': 0.6206242604304331, 'frobenious_norm_RMC': 441.82225512379597, 'max_norm_RMC': 101.41442270181936}


In [31]:
import pickle
pickle.dump(results_df, open('results_100.pt', 'wb'))

In [32]:
results_df['regret_EW'] = results_df['c_EW'] - results_df['c_opt'] 
results_df['regret_DRMF'] = results_df['c_DRMF'] - results_df['c_opt']
results_df['regret_stable_PCP'] = results_df['c_stable_PCP'] - results_df['c_opt']
results_df['regret_RMC'] = results_df['c_RMC'] - results_df['c_opt']
results_df.describe()

,c_opt,c_EW,frobenious_norm_EW,max_norm_EW,c_DRMF,frobenious_norm_DRMF,max_norm_DRMF,c_stable_PCP,frobenious_norm_stable_PCP,max_norm_stable_PCP,c_RMC,frobenious_norm_RMC,max_norm_RMC,regret_EW,regret_DRMF,regret_stable_PCP,regret_RMC
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.472944,0.520367,169.086737,22.015347,1.151612,250.846385,37.989029,1.188936,295.644711,39.953609,1.290406,476.183149,62.025726,0.047423,0.678669,0.715993,0.817462
std,0.258923,0.282123,80.579259,18.124841,0.578816,138.678758,31.197510,0.433227,160.026454,37.908736,0.526485,523.295036,68.842760,0.042604,0.408918,0.318376,0.418779
min,0.025848,0.026298,45.879496,5.166520,0.051492,55.398899,6.660600,0.436338,60.991472,6.157065,0.436338,62.322495,5.803469,0.000208,0.025644,0.200261,0.200261
25%,0.274144,0.303739,102.977954,11.293663,0.747020,132.901145,15.828055,0.882782,168.664036,15.969721,0.884777,185.939840,18.986509,0.016090,0.376416,0.506590,0.511706
50%,0.469201,0.517940,157.584877,15.832826,1.214420,217.928327,31.492340,1.212580,292.856441,29.227633,1.272567,352.864558,36.648197,0.034290,0.656938,0.636147,0.701693
75%,0.662538,0.723988,234.437491,29.344131,1.553875,354.743594,47.119102,1.467493,395.262538,49.950501,1.649915,443.004254,78.019586,0.064086,0.843763,0.891672,1.097260
max,1.097913,1.109913,355.441628,133.360674,2.421523,681.414296,209.801946,2.216998,819.901724,278.275834,2.640284,3235.536257,363.439038,0.200987,1.793392,1.752383,1.972574


In [14]:
results_df.describe()

,c_opt,c_EW,frobenious_norm_EW,max_norm_EW,c_DRMF,frobenious_norm_DRMF,max_norm_DRMF
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.044665,0.049197,167.778164,21.247280,0.103951,245.886330,36.871622
std,0.027357,0.029902,79.847026,15.616014,0.060057,149.562734,29.738591
min,0.000963,0.000999,40.013573,3.846638,0.001985,52.460070,4.658152
25%,0.020920,0.022147,102.601671,11.415754,0.053323,127.222262,15.691043
50%,0.041009,0.046024,159.863287,16.626845,0.106401,205.300977,28.414506
75%,0.066356,0.070829,223.309126,26.162798,0.149411,336.645419,47.231336
max,0.107216,0.108850,431.552288,89.471546,0.218188,720.549030,163.918221
